In [1]:
import scipy.ndimage.interpolation as interp
from astropy.stats import mad_std
from astropy.stats import sigma_clip
from photutils.utils import calc_total_error
import astropy.stats as stat
from photutils import aperture_photometry, CircularAperture, CircularAnnulus, DAOStarFinder
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.visualization import ZScaleInterval
from astropy.io import fits 
import os 
import glob 
import scipy.ndimage.interpolation as interp
from tqdm import tqdm
from tqdm import tqdm
from importlib import reload
from dred import Calibration
import dred

# The Data Reduction Module - DRED

In this notebook, I outline the steps for reducing astronomical data collected with the half degree imager. This notebook utilizes the module 'dred', which can handle the bias subtraction and flatfielding steps in the reduction process. Before using dred, ensure that the paths to your data are all hardcoded based on data type e.g. a path for biases and a path for flats. Once this step is completed, you can begin the reduction process by creating an instance of the class 'Calibration'. Once the user specifies the type of calibration frames they'll be dealing with, they can create a 'master' or median combination of those frames, and apply it to their data. **Note, that the master created is different depending on the calibration frame type.**

### Grouping all the Data into their Respective Folders:

In [2]:
#Getting all the files
all_filenames = glob.glob("Checkin1_BiasFlats/*.fits")
all_files = [f.split('/')[-1] for f in all_filenames]

#Sorting the files by type
for f in all_files:
    dred.filesorter(f, 'Flats', 'Twilight Flat V', 'OBJECT', current_path = "Checkin1_BiasFlats")
    dred.filesorter(f, 'Orion_V', '+05', 'RASTRNG', current_path = "Checkin1_BiasFlats")
    dred.filesorter(f, 'Bias', 'Bias Frame', 'OBJECT', current_path = "Checkin1_BiasFlats")
    
#Set your own file paths
path_to_flats = glob.glob('Checkin1_BiasFlats/Flats/*.fits')

path_to_bias = glob.glob('Checkin1_BiasFlats/Bias/*.fits')

path_to_orion = glob.glob('Checkin1_BiasFlats/Orion_V/*.fits')

## Bias Subtraction

In order to bias subtract our data, we must first create an instance of Calibration. In this example, I call that instance bias_calibration. Using generate_master with a 'bias' calibration type will create a master bias that is the median combination of all the individual bias frames (input as a list). When we "apply" it to a list of science and flat frames, an overscan value is computed for each frame in the input list, and the master bias is scaled accordingly for each image.

In [3]:
#Creating an instance of calibration for bias'
bias_calibration = Calibration('bias')

#Generating a master bias (median combination)
bias_calibration.generate_master(path_to_bias)

#Bias subtract the master bias frame from science images and flats (accounting for overscan)
bias_calibration.apply(path_to_orion, save = True, output_dir = 'Checkin1_BiasFlats/Orion_V/')
bias_calibration.apply(path_to_flats, save = True, output_dir = 'Checkin1_BiasFlats/Flats/')

## Flatfielding

Now that we have our bias subtracted flats and science images, we can use the same methods to now flatfield the data. This time when 'generate_master' is called, it will create create a median combination of all the input flat frames, but alse normalize that median combined flat by its median pixel value, resulting in a ***different*** master frame. Then, when we apply the master flat frame to the data, it will divide all the input science frames by the master flat. 

In [8]:
# Grabbing all the bias subtracted science frames
b_sub_flats = glob.glob('Checkin1_BiasFlats/Flats/b_*.fits')
b_sub_orion = glob.glob('Checkin1_BiasFlats/Orion_V/b_*.fits')

In [4]:
#Flatfielding
flat_calibration = Calibration('flats')
flat_calibration.generate_master(b_sub_flats)

In [9]:
flat_calibration.apply(b_sub_orion, save = True, output_dir = 'Checkin1_BiasFlats/Orion_V/')

UnboundLocalError: local variable 'create_calibrated_image' referenced before assignment